In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

from sklearn.metrics import f1_score,  recall_score, precision_score, accuracy_score

import joblib

np.random.seed(1)
tf.random.set_seed(1)

Tomamos la misma clasificacion que el archivo del chad, para nuestros datos. 

In [18]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_con_tipo_precio_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_con_tipo_precio_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

In [19]:
viviendas

,start_date,end_date,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,geometry,pxm2,tipo_precio
0,2021-12-15,2021-12-25,-34.555049,-58.490513,Saavedra,Departamento,2.0,1.0,58.0,49.0,167113.0,Departamento - Saavedra,POINT (-58.4905135 -34.5550487),2881.258621,Alto
1,2021-05-20,2021-06-05,-34.546991,-58.478553,Saavedra,Departamento,2.0,1.0,48.0,44.0,129000.0,Departamento - Saavedra,POINT (-58.4785532 -34.546991),2687.500000,Medio
2,2021-03-20,2021-04-16,-34.570842,-58.491805,Villa Urquiza,Departamento,2.0,1.0,62.0,58.0,124600.0,Venta Depto 2 amb Anticipo + Cuotas CAC V. Urq...,POINT (-58.4918048 -34.5708421),2009.677419,Medio
3,2021-07-17,2021-10-16,-34.589251,-58.402106,Recoleta,Departamento,3.0,2.0,60.0,60.0,196000.0,Excelente 3 ambientes en venta en Barrio Recoleta,POINT (-58.4021058 -34.5892511),3266.666667,Alto
4,2021-07-02,2022-02-09,-34.626290,-58.437632,Caballito,Departamento,3.0,2.0,67.0,61.0,235000.0,VENTA DEPARTAMENTO 3 AMB. EN CABALLITO,POINT (-58.4376325 -34.6262899),3507.462687,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18571,2021-02-13,2021-03-01,-34.594510,-58.413521,Barrio Norte,Departamento,3.0,2.0,60.0,51.0,148500.0,VENTA DEPARTAMENTO 3 AMBIENTES BALCÓN FRENTE 3...,POINT (-58.413521 -34.59451),2475.000000,Medio
18572,2021-08-20,2200-12-31,-34.540767,-58.466694,Nuñez,Casa,4.0,3.0,210.0,210.0,590000.0,Casa - Nuñez,POINT (-58.4666936 -34.5407667),2809.523810,Alto
18573,2021-09-08,2021-10-14,-34.586723,-58.402579,Recoleta,Departamento,2.0,1.0,35.0,35.0,97000.0,Recoleta 2 amb Oportunidad,POINT (-58.4025792 -34.5867234),2771.428571,Medio
18574,2021-01-12,2021-06-05,-34.579607,-58.489898,Villa Urquiza,Departamento,4.0,3.0,76.0,73.0,245000.0,4 ambientes al contrafrente con balcón y coche...,POINT (-58.4898982 -34.579607),3223.684211,Alto


In [20]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"]) 
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])
viviendas = pd.get_dummies(viviendas, columns = ["tipo_precio"])

In [21]:
viviendas_train, viviendas_test = train_test_split(viviendas, test_size=0.2, random_state=2)

In [22]:
variables_a_predecir = ['tipo_precio_Alto', 'tipo_precio_Bajo', 'tipo_precio_Medio']

In [23]:
columnas_predictoras=viviendas_train.columns.to_list()
variables_a_remover = ['property_price', 'start_date', 'end_date', 'property_title', 'geometry', 'pxm2']
variables_a_remover.extend(variables_a_predecir)
for variable in variables_a_remover:
    columnas_predictoras.remove(variable)
#columnas_predictoras

In [24]:
class ModeloRedes:
    def __init__(self, dataset_train, dataset_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        #self.cantidad_capas = cantidad_capas
        #self.neuronas_por_capa = neuronas_por_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = dataset_test.loc[:,variables_predictoras]
        self.y_test = dataset_test.loc[:,variables_a_predecir]
        self.x_train = dataset_train.loc[:,variables_predictoras]
        self.y_train = dataset_train.loc[:,variables_a_predecir]
        self.x_test_original = None
        self.x_train_original = None
        self.escalar_datasets()
        self.entrenar_modelo()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        
        self.x_train_original = self.x_train
        self.x_test_original = self.x_test
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))

        #self.y_train = keras.utils.to_categorical(self.y_train, 3)
        #self.y_test = keras.utils.to_categorical(self.y_test, 3)



    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=3

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,4,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))
        
        """for i in range(self.cantidad_capas):
            capas.append(keras.layers.Dense(self.neuronas_por_capa, activation= self.funcion_activacion ))"""

        capas.append(keras.layers.Dense(d_out, activation= 'softmax'))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss='categorical_crossentropy', 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        y_pred=self.modelo.predict(self.x_test)
        cm = confusion_matrix(self.y_test,y_pred)
        labels = ['Bajo', 'Alto', 'Medio']
        sns.heatmap(cm, cmap='Blues',annot=True,fmt='g', xticklabels=labels, yticklabels=labels).set(title="Heatmap con set de datos de entrenamiento como entrada")
        plt.xlabel('Prediccion')
        plt.ylabel('Realidad')
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)

        for i in y_pred:
            pos_maximo = 0
            for j in range(len(i)):
                if i[j] > i[pos_maximo]:
                    pos_maximo = j
            for j in range(len(i)):
                if j == pos_maximo:
                    i[j] = 1
                else:
                    i[j]= 0

        self.resultado = f1_score(self.y_test.values.tolist(),y_pred, average='weighted')

        return self.resultado
    
    """def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        y_pred=self.modelo.predict(df_para_predecir)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo

        mse=mean_squared_error(df_precios,y_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return y_pred , resultado"""
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n')

In [25]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)       

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

                                        #vimos que los modelos buenos utilizaban "relu" asique dejamos de utilizar el resto dentro de random search
posibles_funcion_activacion = ['softmax',"relu","tanh", "sigmoid"]
posibles_metricas = ['AUC', 'accuracy', 'Precision', 'Recall']

posibles_learning_rate = [0.1, 0.01]

In [26]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

def funcion_optimizadoraSGD(learnigRate):
            return keras.optimizers.SGD(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA, funcion_optimizadoraSGD]

In [27]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

In [28]:
def random_validator(cantidad_modelos, posibles_funcion_activacion, posibles_optimizador, 
    posibles_learning_rate, posibles_metricas, obtener_posibles_variables_predictoras):
    
    resultados = []
    for i in range(cantidad_modelos):

        variables_predictoras = obtener_posibles_variables_predictoras()
        cantidad_de_primera_capa = random.randrange(4, 20, 1)
        """cantidad_de_capas = random.randrange(1, 10, 1)
        neuronas_por_capa = random.randrange(2, 300, 1)"""

        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        df_reducido = viviendas_train[:3000]#random.randrange(3000, 5000, 1)
        df_train, df_test = train_test_split(df_reducido.loc[:],test_size=0.2)

        print(variables_predictoras)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        #cantidad_de_capas, '\n',
        #neuronas_por_capa, '\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',)

        modelo = ModeloRedes(df_train, df_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [29]:
#Hace 50 modelos con hiperparametros random random
def func():
    resultados = random_validator(50, posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

    return resultados

In [30]:
#dado un conjunto de modelos se queda con los modelos cuyo rmse sea menor a 130000
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] > 0.6 ):
            mejores.append(i)

    return mejores

In [31]:
mejores = []

In [32]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_covered', 'barrio_Mataderos', 'barrio_Barracas', 'barrio_Colegiales', 'barrio_Puerto Madero', 'barrio_Abasto', 'barrio_Barrio Norte', 'barrio_Congreso', 'barrio_Parque Centenario', 'barrio_Caballito', 'barrio_Pompeya', 'barrio_Balvanera', 'barrio_Agronomía', 'barrio_Tribunales', 'barrio_Recoleta', 'barrio_Villa del Parque', 'property_rooms', 'barrio_Retiro', 'barrio_Villa Real', 'barrio_Parque Chacabuco', 'barrio_Velez Sarsfield', 'barrio_Villa Ortuzar', 'barrio_Flores', 'barrio_San Telmo', 'barrio_Boedo', 'barrio_Once', 'barrio_Chacarita', 'barrio_Villa Santa Rita', 'barrio_Coghlan', 'barrio_Paternal', 'barrio_Villa Luro']

 

0 
 ['property_surface_covered', 'barrio_Mataderos', 'barrio_Barracas', 'barrio_Colegiales', 'barrio_Puerto Madero', 'barrio_Abasto', 'barrio_Barrio Norte', 'barrio_Congreso', 'barrio_Parque Centenario', 'barrio_Caballito', 'barrio_Pompeya', 'barrio_Balvanera', 'barrio_Agronomía', 'barrio_Tribunales', 'barrio_Recoleta', 'barrio_Villa del Parqu

In [33]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'longitud', 'barrio_Congreso', 'barrio_Centro / Microcentro', 'barrio_Palermo', 'barrio_Barrio Norte', 'barrio_Flores', 'property_bedrooms', 'barrio_Villa Real', 'barrio_Parque Patricios', 'barrio_Caballito', 'barrio_Abasto']

 

0 
 ['latitud', 'longitud', 'barrio_Congreso', 'barrio_Centro / Microcentro', 'barrio_Palermo', 'barrio_Barrio Norte', 'barrio_Flores', 'property_bedrooms', 'barrio_Villa Real', 'barrio_Parque Patricios', 'barrio_Caballito', 'barrio_Abasto'] 
 12 
 softmax 
 <function funcion_optimizadoraNADAM at 0x0000014E50F6CE50> 
 0.1 
 accuracy 
 240 
 619 

19/19 [==============================] - 0s 679us/step
0.3314831294030405
['property_surface_covered', 'latitud', 'property_rooms', 'barrio_Boedo', 'barrio_Villa Urquiza', 'barrio_San Cristobal', 'barrio_Constitución']

 

1 
 ['property_surface_covered', 'latitud', 'property_rooms', 'barrio_Boedo', 'barrio_Villa Urquiza', 'barrio_San Cristobal', 'barrio_Constitución'] 
 7 
 relu 
 <function funcion_optimi

In [34]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'barrio_Villa Real', 'barrio_Agronomía', 'barrio_Barracas', 'barrio_San Nicolás', 'barrio_Parque Patricios', 'barrio_Boca', 'barrio_Villa General Mitre', 'barrio_Villa Soldati', 'property_bedrooms', 'barrio_Belgrano', 'barrio_Villa Luro', 'barrio_Monserrat', 'barrio_Villa Pueyrredón', 'barrio_Villa Santa Rita', 'barrio_Villa del Parque', 'barrio_Mataderos', 'property_rooms', 'barrio_Parque Chas', 'barrio_Saavedra', 'barrio_Chacarita', 'barrio_Monte Castro', 'barrio_Versalles', 'barrio_Retiro', 'barrio_Barrio Norte', 'barrio_Liniers', 'barrio_Parque Avellaneda', 'property_type_Departamento', 'barrio_Palermo', 'barrio_Villa Devoto', 'barrio_Coghlan', 'barrio_Once', 'barrio_Centro / Microcentro', 'barrio_Catalinas', 'barrio_Balvanera', 'barrio_Puerto Madero', 'barrio_Paternal', 'barrio_Constitución', 'barrio_Villa Ortuzar', 'barrio_San Telmo']

 

0 
 ['property_surface_total', 'barrio_Villa Real', 'barrio_Agronomía', 'barrio_Barracas', 'barrio_San Nicolás', 'ba

In [35]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_covered', 'latitud', 'property_surface_total', 'longitud', 'barrio_Villa Devoto', 'property_type_PH', 'barrio_Catalinas', 'barrio_Parque Patricios', 'barrio_Chacarita', 'barrio_Parque Avellaneda', 'barrio_Congreso', 'barrio_Liniers', 'barrio_Parque Centenario', 'barrio_Puerto Madero', 'barrio_Las Cañitas', 'barrio_Villa Ortuzar', 'barrio_Agronomía', 'barrio_Belgrano', 'barrio_Villa Santa Rita', 'barrio_Barracas', 'barrio_Flores', 'barrio_Versalles', 'barrio_Almagro', 'barrio_Villa Real', 'barrio_Villa Crespo']

 

0 
 ['property_surface_covered', 'latitud', 'property_surface_total', 'longitud', 'barrio_Villa Devoto', 'property_type_PH', 'barrio_Catalinas', 'barrio_Parque Patricios', 'barrio_Chacarita', 'barrio_Parque Avellaneda', 'barrio_Congreso', 'barrio_Liniers', 'barrio_Parque Centenario', 'barrio_Puerto Madero', 'barrio_Las Cañitas', 'barrio_Villa Ortuzar', 'barrio_Agronomía', 'barrio_Belgrano', 'barrio_Villa Santa Rita', 'barrio_Barracas', 'barrio_Flores', 'bar

In [36]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['longitud', 'property_surface_total', 'latitud', 'barrio_Palermo', 'property_type_Departamento', 'barrio_Villa Devoto', 'barrio_Parque Chas']

 

0 
 ['longitud', 'property_surface_total', 'latitud', 'barrio_Palermo', 'property_type_Departamento', 'barrio_Villa Devoto', 'barrio_Parque Chas'] 
 9 
 tanh 
 <function funcion_optimizadoraSGD at 0x0000014E52C695E0> 
 0.01 
 accuracy 
 180 
 283 

19/19 [==============================] - 0s 716us/step
0.525942621106725
['latitud', 'property_surface_total', 'barrio_Monte Castro', 'barrio_San Nicolás', 'barrio_Parque Patricios', 'barrio_Constitución', 'barrio_Villa Ortuzar', 'property_type_Departamento', 'barrio_Villa Riachuelo', 'barrio_Boedo', 'barrio_Villa Real', 'barrio_Coghlan', 'barrio_Villa del Parque', 'barrio_Catalinas', 'barrio_Centro / Microcentro', 'barrio_Pompeya', 'barrio_Villa Soldati', 'barrio_Barrio Norte', 'barrio_Saavedra', 'barrio_Parque Chas', 'barrio_Monserrat', 'barrio_Barracas', 'barrio_Once', 'barrio_San Telmo', 'barr

In [37]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['longitud', 'latitud', 'barrio_Once', 'barrio_Boedo', 'barrio_Colegiales', 'barrio_Villa Crespo', 'barrio_Palermo', 'barrio_Caballito', 'barrio_Pompeya', 'barrio_Las Cañitas', 'barrio_Tribunales', 'barrio_Monte Castro', 'property_type_Casa', 'barrio_Liniers', 'barrio_Recoleta', 'barrio_Boca', 'property_type_PH', 'barrio_Parque Avellaneda', 'barrio_San Cristobal']

 

0 
 ['longitud', 'latitud', 'barrio_Once', 'barrio_Boedo', 'barrio_Colegiales', 'barrio_Villa Crespo', 'barrio_Palermo', 'barrio_Caballito', 'barrio_Pompeya', 'barrio_Las Cañitas', 'barrio_Tribunales', 'barrio_Monte Castro', 'property_type_Casa', 'barrio_Liniers', 'barrio_Recoleta', 'barrio_Boca', 'property_type_PH', 'barrio_Parque Avellaneda', 'barrio_San Cristobal'] 
 7 
 softmax 
 <function funcion_optimizadoraRMSPROP at 0x0000014E52C693A0> 
 0.1 
 Precision 
 160 
 476 

19/19 [==============================] - 0s 610us/step
0.4771743942371315
['latitud', 'property_surface_covered', 'property_surface_total', 'longitud

In [38]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'barrio_Retiro', 'barrio_Paternal', 'barrio_Monserrat', 'barrio_Villa Pueyrredón', 'barrio_Nuñez']

 

0 
 ['property_surface_total', 'barrio_Retiro', 'barrio_Paternal', 'barrio_Monserrat', 'barrio_Villa Pueyrredón', 'barrio_Nuñez'] 
 16 
 relu 
 <function funcion_optimizadoraADAMAX at 0x0000014E52C694C0> 
 0.1 
 Precision 
 150 
 226 

19/19 [==============================] - 0s 594us/step
0.35211371496287697
['longitud', 'property_surface_covered', 'latitud', 'barrio_Retiro', 'barrio_San Nicolás', 'property_type_Casa', 'barrio_Parque Centenario', 'barrio_Almagro', 'barrio_Parque Patricios', 'property_rooms', 'barrio_Flores', 'barrio_Villa Devoto', 'barrio_Villa Santa Rita', 'property_type_Departamento', 'barrio_Villa del Parque', 'barrio_Villa Luro', 'barrio_Villa Crespo', 'barrio_Paternal', 'barrio_Once', 'barrio_Villa General Mitre', 'barrio_Villa Pueyrredón', 'barrio_Villa Urquiza', 'barrio_Constitución', 'barrio_Tribunales', 'barrio_Abasto', 'barrio_San

In [39]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'latitud', 'property_surface_covered', 'longitud', 'barrio_Villa Lugano', 'barrio_Boedo', 'barrio_Villa Ortuzar', 'barrio_Versalles', 'barrio_Abasto', 'barrio_Villa Santa Rita', 'barrio_Liniers', 'barrio_Retiro', 'barrio_Pompeya', 'barrio_Once', 'barrio_Velez Sarsfield', 'barrio_Villa Urquiza', 'barrio_Villa Riachuelo', 'barrio_Villa Real', 'barrio_Villa del Parque', 'barrio_Floresta', 'barrio_Nuñez', 'barrio_Belgrano', 'property_type_Departamento', 'barrio_Barrio Norte', 'barrio_Saavedra', 'barrio_Parque Avellaneda', 'barrio_Monserrat', 'barrio_Centro / Microcentro', 'property_type_Casa', 'barrio_Almagro', 'barrio_San Nicolás', 'barrio_Boca', 'barrio_Paternal', 'property_rooms', 'barrio_Parque Patricios', 'barrio_Parque Chas', 'barrio_Recoleta', 'barrio_Villa Luro', 'barrio_Las Cañitas', 'barrio_Mataderos', 'barrio_Catalinas', 'barrio_Balvanera', 'barrio_Villa Soldati', 'barrio_Villa Pueyrredón', 'barrio_Parque Chacabuco', 'barrio_Coghlan', 'barrio_Chacarita

In [40]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'barrio_Palermo', 'property_type_PH', 'barrio_Catalinas', 'barrio_Almagro', 'barrio_Mataderos', 'barrio_Boca', 'barrio_Barracas', 'barrio_Belgrano', 'barrio_Villa Soldati', 'barrio_Villa Real', 'barrio_Recoleta', 'barrio_Barrio Norte', 'barrio_Villa Devoto', 'barrio_Villa General Mitre', 'barrio_San Telmo', 'barrio_San Nicolás', 'barrio_Colegiales', 'barrio_Congreso', 'barrio_Agronomía', 'barrio_Parque Chacabuco', 'barrio_Versalles', 'property_type_Departamento', 'barrio_Parque Centenario', 'barrio_San Cristobal', 'barrio_Villa Lugano', 'barrio_Once', 'barrio_Parque Chas', 'barrio_Pompeya']

 

0 
 ['latitud', 'barrio_Palermo', 'property_type_PH', 'barrio_Catalinas', 'barrio_Almagro', 'barrio_Mataderos', 'barrio_Boca', 'barrio_Barracas', 'barrio_Belgrano', 'barrio_Villa Soldati', 'barrio_Villa Real', 'barrio_Recoleta', 'barrio_Barrio Norte', 'barrio_Villa Devoto', 'barrio_Villa General Mitre', 'barrio_San Telmo', 'barrio_San Nicolás', 'barrio_Colegiales', 'barrio_Congreso',

In [41]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'property_surface_covered', 'longitud', 'barrio_Barracas', 'barrio_Boedo', 'barrio_Pompeya', 'barrio_Flores', 'barrio_Nuñez', 'barrio_Versalles', 'barrio_Almagro', 'barrio_Parque Avellaneda', 'barrio_Monserrat', 'barrio_Saavedra', 'barrio_Velez Sarsfield', 'barrio_Belgrano', 'barrio_Las Cañitas', 'barrio_Liniers', 'barrio_Villa Pueyrredón', 'barrio_Paternal', 'barrio_Boca', 'barrio_Colegiales', 'barrio_Recoleta', 'barrio_Parque Chas', 'barrio_Villa Crespo', 'barrio_Villa Real', 'barrio_Villa Soldati', 'property_type_Departamento', 'barrio_Coghlan', 'barrio_Villa Santa Rita', 'barrio_Puerto Madero', 'barrio_Balvanera', 'barrio_Palermo', 'barrio_Congreso', 'barrio_Constitución', 'barrio_Villa Urquiza', 'barrio_Floresta', 'property_type_PH', 'barrio_Villa Devoto', 'barrio_Villa del Parque', 'barrio_Retiro', 'property_rooms', 'barrio_Catalinas', 'barrio_Abasto', 'property_bedrooms', 'barrio_Chacarita', 'barrio_Caballito', 'barrio_San Cristobal', 'barrio_Monte Castro', 'barrio_C

In [42]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['longitud', 'barrio_San Telmo', 'barrio_Parque Chacabuco', 'barrio_Catalinas', 'barrio_Villa Luro', 'barrio_Caballito', 'barrio_Villa Soldati', 'barrio_Parque Chas', 'barrio_Balvanera', 'barrio_Liniers', 'barrio_Villa Real', 'barrio_Centro / Microcentro', 'barrio_Almagro', 'barrio_San Nicolás', 'barrio_Parque Centenario', 'property_type_Casa', 'barrio_Flores', 'barrio_Pompeya', 'barrio_Puerto Madero', 'barrio_San Cristobal', 'barrio_Monserrat', 'barrio_Las Cañitas', 'barrio_Velez Sarsfield', 'barrio_Abasto', 'barrio_Paternal', 'barrio_Coghlan', 'barrio_Parque Patricios', 'barrio_Constitución', 'barrio_Boca', 'barrio_Recoleta', 'barrio_Boedo', 'barrio_Villa Riachuelo', 'barrio_Mataderos', 'barrio_Once', 'barrio_Agronomía', 'barrio_Floresta', 'barrio_Palermo', 'barrio_Parque Avellaneda', 'barrio_Villa Devoto', 'barrio_Versalles', 'barrio_Villa del Parque', 'barrio_Villa Pueyrredón', 'barrio_Congreso', 'barrio_Villa Crespo', 'property_rooms', 'barrio_Villa General Mitre', 'barrio_Belgran

In [43]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'property_surface_covered', 'longitud', 'property_surface_total', 'barrio_Las Cañitas', 'barrio_Parque Chas', 'barrio_Belgrano', 'barrio_Monte Castro', 'property_type_PH', 'property_type_Casa', 'barrio_Villa Real', 'barrio_Villa Lugano', 'barrio_Flores', 'barrio_Abasto', 'barrio_Recoleta', 'barrio_Colegiales', 'barrio_Coghlan', 'barrio_Villa Riachuelo']

 

0 
 ['latitud', 'property_surface_covered', 'longitud', 'property_surface_total', 'barrio_Las Cañitas', 'barrio_Parque Chas', 'barrio_Belgrano', 'barrio_Monte Castro', 'property_type_PH', 'property_type_Casa', 'barrio_Villa Real', 'barrio_Villa Lugano', 'barrio_Flores', 'barrio_Abasto', 'barrio_Recoleta', 'barrio_Colegiales', 'barrio_Coghlan', 'barrio_Villa Riachuelo'] 
 10 
 sigmoid 
 <function funcion_optimizadoraSGD at 0x0000014E52C695E0> 
 0.01 
 AUC 
 250 
 931 

19/19 [==============================] - 0s 2ms/step
0.3277926421404682
['longitud', 'property_surface_total', 'property_surface_covered', 'latitud', 'barr

In [44]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'barrio_Parque Patricios', 'barrio_Saavedra']

 

0 
 ['property_surface_total', 'barrio_Parque Patricios', 'barrio_Saavedra'] 
 5 
 relu 
 <function funcion_optimizadoraSGD at 0x0000014E52C695E0> 
 0.01 
 accuracy 
 260 
 358 

19/19 [==============================] - 0s 457us/step
0.38648177174574494
['latitud', 'property_surface_total', 'barrio_Paternal', 'property_rooms', 'barrio_Catalinas', 'barrio_Liniers', 'barrio_Monte Castro', 'barrio_Barracas', 'barrio_Almagro', 'barrio_Parque Avellaneda', 'barrio_Balvanera', 'barrio_Congreso', 'barrio_Villa Real', 'barrio_Parque Chas', 'barrio_Nuñez', 'barrio_Parque Chacabuco', 'barrio_Caballito', 'barrio_San Nicolás', 'barrio_Villa Lugano', 'barrio_Saavedra', 'barrio_Agronomía', 'barrio_Tribunales', 'property_type_PH', 'barrio_Las Cañitas', 'barrio_Villa Crespo', 'barrio_Abasto', 'barrio_Floresta', 'barrio_Constitución', 'barrio_Once', 'barrio_Colegiales', 'barrio_Villa Devoto', 'barrio_Villa del Parque', 'barrio_

In [45]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'latitud', 'longitud', 'barrio_Belgrano', 'barrio_Villa Riachuelo', 'barrio_San Telmo', 'barrio_Villa Soldati', 'barrio_Monte Castro', 'property_type_Departamento', 'barrio_Parque Chas', 'barrio_Villa Lugano', 'barrio_Retiro', 'property_bedrooms', 'barrio_Almagro', 'barrio_Parque Avellaneda', 'barrio_Floresta', 'barrio_Puerto Madero', 'barrio_Versalles', 'barrio_Boedo', 'property_rooms', 'barrio_Flores', 'barrio_Congreso', 'barrio_Villa Crespo', 'barrio_Boca', 'barrio_Parque Centenario', 'barrio_Pompeya', 'barrio_Balvanera', 'barrio_Catalinas', 'property_type_Casa', 'barrio_Constitución', 'barrio_Once', 'barrio_Villa General Mitre', 'property_type_PH', 'barrio_Tribunales']

 

0 
 ['property_surface_total', 'latitud', 'longitud', 'barrio_Belgrano', 'barrio_Villa Riachuelo', 'barrio_San Telmo', 'barrio_Villa Soldati', 'barrio_Monte Castro', 'property_type_Departamento', 'barrio_Parque Chas', 'barrio_Villa Lugano', 'barrio_Retiro', 'property_bedrooms', 'barrio_

In [46]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'property_surface_covered', 'property_surface_total', 'longitud', 'barrio_Velez Sarsfield', 'barrio_Monte Castro', 'barrio_Floresta', 'property_bedrooms', 'property_type_Departamento', 'barrio_Centro / Microcentro', 'barrio_Congreso', 'barrio_Almagro', 'barrio_Villa Crespo', 'barrio_Abasto', 'barrio_Parque Patricios', 'barrio_Villa Riachuelo', 'barrio_Villa General Mitre', 'barrio_Pompeya', 'barrio_Parque Centenario']

 

0 
 ['latitud', 'property_surface_covered', 'property_surface_total', 'longitud', 'barrio_Velez Sarsfield', 'barrio_Monte Castro', 'barrio_Floresta', 'property_bedrooms', 'property_type_Departamento', 'barrio_Centro / Microcentro', 'barrio_Congreso', 'barrio_Almagro', 'barrio_Villa Crespo', 'barrio_Abasto', 'barrio_Parque Patricios', 'barrio_Villa Riachuelo', 'barrio_Villa General Mitre', 'barrio_Pompeya', 'barrio_Parque Centenario'] 
 15 
 relu 
 <function funcion_optimizadoraADAMAX at 0x0000014E52C694C0> 
 0.1 
 AUC 
 190 
 679 

19/19 [=================

In [47]:
mejores

[[],
 [],
 [],
 [[<__main__.ModeloRedes at 0x14efae3d1c0>, 0.6001097305819932]],
 [[<__main__.ModeloRedes at 0x14e56e7bcd0>, 0.6141855339928969]],
 [[<__main__.ModeloRedes at 0x14e9ff5e520>, 0.6286353447498018]],
 [[<__main__.ModeloRedes at 0x14e5768e670>, 0.6149057503850761]],
 [[<__main__.ModeloRedes at 0x14e87529280>, 0.6096916292398743]],
 [[<__main__.ModeloRedes at 0x14e87ab27c0>, 0.6097376953591698]],
 [[<__main__.ModeloRedes at 0x14efc231b20>, 0.6308853780435804],
  [<__main__.ModeloRedes at 0x14e61144610>, 0.6232339647728028]],
 [[<__main__.ModeloRedes at 0x14ed5c582b0>, 0.6031968444228704]],
 [[<__main__.ModeloRedes at 0x14ea74f6e20>, 0.604238405298225]],
 [[<__main__.ModeloRedes at 0x14e55228b20>, 0.6013426349861627]],
 [[<__main__.ModeloRedes at 0x14e9e856cd0>, 0.6160512844526477]],
 [[<__main__.ModeloRedes at 0x14e534da850>, 0.6100254920833221],
  [<__main__.ModeloRedes at 0x14e51d317c0>, 0.6079498531540346]]]

In [48]:
#reentrena un modelo con los mismo hiperparametros a ver si con una semilla mejor mejora la metrica
def reentrenar_tot(modelo):
    #modelo.info()
    mejor_modelo = ModeloRedes(
        viviendas_train, 
        viviendas_test, 
        modelo.variables_predictoras, 
        modelo.cantidad_primera_capa, 
        modelo.funcion_activacion, 
        modelo.optimizador, 
        modelo.learning_rate, 
        modelo.metrica, 
        modelo.epoch, 
        modelo.batch_size)
        
    mejor_modelo.testear_modelo()
    mejor_resultado = mejor_modelo.resultado 
   

    for i in range(0):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(
            viviendas_train, 
            viviendas_test, 
            modelo.variables_predictoras, 
            modelo.cantidad_primera_capa, 
            modelo.funcion_activacion, 
            modelo.optimizador, 
            modelo.learning_rate, 
            modelo.metrica, 
            modelo.epoch, 
            modelo.batch_size)

        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        mejor_modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [49]:
tots = []

In [50]:
for i in mejores:
    for j in i:
        tots.append(reentrenar_tot(j[0]))
        print(tots[-1])

581/581 [==============================] - 0s 701us/step
[<__main__.ModeloRedes object at 0x0000014EE2720490>, 0.6040880247243161]
581/581 [==============================] - 0s 640us/step
[<__main__.ModeloRedes object at 0x0000014E88CA5F70>, 0.6185828155805074]
581/581 [==============================] - 0s 444us/step
[<__main__.ModeloRedes object at 0x0000014EBFFE7FA0>, 0.6296601299742202]
581/581 [==============================] - 1s 927us/step
[<__main__.ModeloRedes object at 0x0000014E88D1AC10>, 0.6348748246541656]
581/581 [==============================] - 0s 716us/step
[<__main__.ModeloRedes object at 0x0000014E68A41E20>, 0.6349618008334542]
581/581 [==============================] - 1s 1ms/step
[<__main__.ModeloRedes object at 0x0000014EBD8C6400>, 0.612711984318291]
581/581 [==============================] - 1s 894us/step
[<__main__.ModeloRedes object at 0x0000014EC9F4E040>, 0.6324073503689829]
581/581 [==============================] - 0s 699us/step
[<__main__.ModeloRedes object

In [51]:
mejor_modelo = tots[0]
for i in tots:
    if (i != None):
        if (i[1] > mejor_modelo[1]):
            mejor_modelo = i

In [52]:
mejor_modelo

[<__main__.ModeloRedes at 0x14efc1d1250>, 0.6423362220014759]

In [53]:
joblib.dump(mejor_modelo[0], './Models/clasif_ultima_tanda') 

INFO:tensorflow:Assets written to: ram://51d71c25-c817-42d6-8271-4c40cd7329bb/assets


['./Models/clasif_ultima_tanda']